In [ ]:
import os

from cartopy import crs
import distributed
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr

_ = xr.set_options(keep_attrs=True)

In [ ]:
# use a large filesystem for cahce
#
# os.chdir("/data")

# activate dask distributed
#
# client = distributed.Client("tcp://dask-scheduler:8786")
# client

In [ ]:
# !pip install ecmwf-opendata 
# !pip install -e ../

In [ ]:
request = {
    "dataset": "seasonal-original-single-levels",
    "originating_centre": 'ecmwf',
    "system": '51',
    "variable": ["2m_temperature"],
    "year": ["2023"],
    "month": ["09"],
    "day": ["01"],
    "time": ["00:00"],
    "leadtime_hour": [
        "180", "360", "540",
    ],
}

In [ ]:
%%time

ds = xr.open_dataset(
    request,
    engine="ecmwf",
    request_chunks={"step": 1},
    cache_kwargs={"cache_file": True},
    chunks={},
)
ds

In [ ]:
# visualisation

italy = dict(lat=slice(35, 47), lon=slice(6, 19))
vmax = 0.13
projection = crs.EuroPP()
cmap = "Blues"

In [ ]:
ds.data_vars["2m_temperature"].isel(step=1)

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.data_vars["2m_temperature"].isel(step=2, number=1, latitude=slice(None, None, -1)).plot(ax=ax, transform=crs.PlateCarree())
_ = ax.coastlines()

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.data_vars["2m_temperature"].mean(("step", "number")).sel(latitude=slice(None, None, -1)).plot(ax=ax, transform=crs.PlateCarree())
_ = ax.coastlines()